In [1]:
import pandas as pd
import pprint

# Clustering tool data fetching and processing

Work order **2100058590** / **2100024286** are usable as an example

## Useful functions

In [2]:
def read_csv_file(file_name, columns_dict):
    df =  pd.read_csv('./data/E1P_010_' + file_name + '.csv', delimiter = ';', usecols = columns_dict.keys())
    return df.rename(columns=columns_dict)

def write_mid_level_csv_file(df, file_name):
    df.to_csv('./data/mid_' + file_name + '.csv' if '.csv' not in file_name else '')

## 1. Work Center tables


Tables:
- CHRD
- INOB
- AUSP

### Read needed tables

#### Read CRHD data

Work center header data

In [3]:
# Needed columns
crhd_columns = {
    'OBJID': 'CRHD_WBS_ID',
    'ARBPL': 'CRHD_WBS_Name',
    'WERKS': 'CRHD_WBS_Plant'
}

# Read CRHD table
crhd_df = read_csv_file("CRHD_V1", crhd_columns)

# Build the WBS name column
crhd_df["CRHD_WBS_Full_Name"] = crhd_df["CRHD_WBS_Plant"] + crhd_df["CRHD_WBS_Name"]

print(crhd_df)


     CRHD_WBS_ID CRHD_WBS_Name CRHD_WBS_Plant CRHD_WBS_Full_Name
0       10001014      DRILLING           DK10       DK10DRILLING
1       10001016      WELLMAIN           DK10       DK10WELLMAIN
2       10001017      WELLSUPV           DK10       DK10WELLSUPV
3       10001018      WELLTECH           DK10       DK10WELLTECH
4       10001019      MAINONSH           DK10       DK10MAINONSH
..           ...           ...            ...                ...
405     10001445      MTN-ROPE           DK90       DK90MTN-ROPE
406     10001424      VEN-INST           DK90       DK90VEN-INST
407     10001431      INP-SITE           DK90       DK90INP-SITE
408     10001978      VEN-TURB           DK90       DK90VEN-TURB
409     10001974      MTN-PIPF           DK90       DK90MTN-PIPF

[410 rows x 4 columns]


#### Read INOB data

Work center type 

In [4]:
# Needed columns
inob_columns = {
    'CUOBJ': 'INOB_Object_Number',
    'KLART': 'INOB_Class_Type',
    'OBJEK': 'INOB_Object_Key'
}

# Read INOB table
inob_df = read_csv_file('INOB', inob_columns)

# Filter on KLART == 19
inob_df = inob_df[inob_df['INOB_Class_Type'] == 19]
# Filter on INOB_Object_Key starting with DK
inob_df = inob_df[inob_df['INOB_Object_Key'].str.startswith("DK")]

print(inob_df)

C:\Users\L0551542\AppData\Local\Temp\ipykernel_24412\1343571107.py:2: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df =  pd.read_csv('./data/E1P_010_' + file_name + '.csv', delimiter = ';', usecols = columns_dict.keys())


         INOB_Object_Number  INOB_Class_Type INOB_Object_Key
1153422             2588124               19    DK90MTN-PIPF
1153423             2588126               19    DK30VEN-TURB
1154884             2588125               19    DK20VEN-TURB
1154885             2588127               19    DK70VEN-TURB
1154886             2588128               19    DK90VEN-TURB
...                     ...              ...             ...
2121029             1663533               19    DK30MTN-TURB
2121030             1663751               19    DK80MTN-PAIN
2123700             1664209               19     DK70CON-NPT
2123701             1664210               19     DK80CON-NPT
2123702             1664211               19     DK90CON-NPT

[410 rows x 3 columns]


#### Read AUSP data

This table contains some objects accessible using a object key, we are here looking for the department order

__

After a deep look, there isn't any work order in denmark getting data from this table

In [5]:
# Needed columns
ausp_columns = {
    'OBJEK': 'AUSP_Object_Number',
    'KLART': 'AUSP_Class_Type',
    'ATWRT': 'AUSP_Description'
}

# Read AUSP table
ausp_df = read_csv_file('AUSP', ausp_columns)

# Filter on KLART == 19
ausp_df = ausp_df[ausp_df['AUSP_Class_Type'] == 19]

ausp_df["AUSP_Object_Number"] = ausp_df["AUSP_Object_Number"].astype(int)

print(ausp_df)

C:\Users\L0551542\AppData\Local\Temp\ipykernel_24412\1343571107.py:2: DtypeWarning: Columns (2,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df =  pd.read_csv('./data/E1P_010_' + file_name + '.csv', delimiter = ';', usecols = columns_dict.keys())


         AUSP_Object_Number AUSP_Class_Type AUSP_Description
1891801              235125              19             ELEC
1891802              235132              19            OTHER
1891887              235137              19            OTHER
1892013              235156              19             INSP
1892014              235182              19            OTHER
...                     ...             ...              ...
5513578               18305              19             FABM
5513579               18306              19            PAINT
5513580               18307              19             SCAF
5513581               18308              19             TELE
5513582               18308              19            OTHER

[1365 rows x 3 columns]


In [6]:
print(ausp_df["AUSP_Description"].unique())

['ELEC' 'OTHER' 'INSP' 'MAINT' 'PROD' 'MECH' 'CONS' 'FABM' 'DRILL' 'INST'
 'TELECOM' 'PERF' 'INTG' 'TELE' 'MNGT' 'PAINT' 'SCAF']


### Merge tables

#### Merge INOB and AUSP

In [7]:
# Merge INOB and AUSP - nothing to merge atm, so nothing to do to not pollute the model
df_work_center_merged = inob_df.merge(ausp_df, how = 'left', left_on = ['INOB_Object_Number'], right_on = ['AUSP_Object_Number'])

#df_work_center_merged = inob_df.copy()
#df_work_center_merged = df_work_center_merged[df_work_center_merged['INOB_Object_Key'].str.startswith('DK')]
print(df_work_center_merged)

print(df_work_center_merged['AUSP_Object_Number'].unique())

     INOB_Object_Number  INOB_Class_Type INOB_Object_Key  AUSP_Object_Number  \
0               2588124               19    DK90MTN-PIPF                 NaN   
1               2588126               19    DK30VEN-TURB                 NaN   
2               2588125               19    DK20VEN-TURB                 NaN   
3               2588127               19    DK70VEN-TURB                 NaN   
4               2588128               19    DK90VEN-TURB                 NaN   
..                  ...              ...             ...                 ...   
405             1663533               19    DK30MTN-TURB                 NaN   
406             1663751               19    DK80MTN-PAIN                 NaN   
407             1664209               19     DK70CON-NPT                 NaN   
408             1664210               19     DK80CON-NPT                 NaN   
409             1664211               19     DK90CON-NPT                 NaN   

    AUSP_Class_Type AUSP_Description  


#### Merge CRHD and df_work_center_merged

In [8]:
# Merge CRHD and df_work_center_merged
df_work_center_merged = df_work_center_merged.merge(crhd_df, how = 'left', left_on = ['INOB_Object_Key'], right_on = ['CRHD_WBS_Full_Name'])

print(df_work_center_merged)

     INOB_Object_Number  INOB_Class_Type INOB_Object_Key  AUSP_Object_Number  \
0               2588124               19    DK90MTN-PIPF                 NaN   
1               2588126               19    DK30VEN-TURB                 NaN   
2               2588125               19    DK20VEN-TURB                 NaN   
3               2588127               19    DK70VEN-TURB                 NaN   
4               2588128               19    DK90VEN-TURB                 NaN   
..                  ...              ...             ...                 ...   
405             1663533               19    DK30MTN-TURB                 NaN   
406             1663751               19    DK80MTN-PAIN                 NaN   
407             1664209               19     DK70CON-NPT                 NaN   
408             1664210               19     DK80CON-NPT                 NaN   
409             1664211               19     DK90CON-NPT                 NaN   

    AUSP_Class_Type AUSP_Description  C

### Filter and rename columns to get the final Work Center merge

Columns kept:
* CRHD_WBS_ID -> WBS_ID
* CRHD_WBS_Name -> WBS_Name
* CRHD_WBS_Plant -> WBS_Plant
* CRHD_WBS_Full_Name -> WBS_Full_name
* AUSP_Description -> WBS_Description

In [9]:
df_work_center_merged = df_work_center_merged[["CRHD_WBS_ID", "CRHD_WBS_Name", "CRHD_WBS_Plant", "CRHD_WBS_Full_Name"]]
df_work_center_merged.rename(inplace = True, columns = {
    "CRHD_WBS_ID": "WBS_ID",
    "CRHD_WBS_Name": "WBS_Name",
    "CRHD_WBS_Plant": "WBS_Plant",
    "CRHD_WBS_Full_Name": "WBS_Full_name",
    "AUSP_Description": "WBS_Description"
})

### Save results in csv file

In [10]:
write_mid_level_csv_file(df_work_center_merged, "work_center")

## 2. Work Order and Operations Status tables

Tables:
- TJ30T
- JSTO
- JEST_OR (based on JEST but for Work Orders only)
- JEST_OV (based on JEST but for Operations only)
- TJ02T

### Read needed tables

#### Read TJ30T data

This table contains all available status in E - not linked to system

To fetch a code, the code of the status must be used **ESTAT** and the profile of the user i.e. the work order **STSMA**

__

This table contains only status in E, not I, displayed in the "Status with Status Number" and "Status Without Status No." tables.

Data for the status contained in the "Syst. Status" table are stored in E1P_010_TJ02T

__

**SPRAS** column is the laguage used, E for english, D for danish... only keep english





In [11]:
# Needed columns
tj30t_columns = {
    'ESTAT': 'TJ30T_User_Status_ID', 
    'STSMA': 'TJ30T_User_Status_Profile',
    'SPRAS': 'TJ30T_User_Language',
    'TXT04': 'TJ30T_Textual_Status',
    'TXT30': 'TJ30T_Long_Textual_Status'
}

# Read TJ30T table
tj30t_df = read_csv_file('TJ30T', tj30t_columns)

# Only keep english
tj30t_df = tj30t_df[tj30t_df["TJ30T_User_Language"] == "E"]

print(tj30t_df)

write_mid_level_csv_file(tj30t_df, "tj30t_filtered")

     TJ30T_User_Status_Profile TJ30T_User_Status_ID TJ30T_User_Language  \
1                     00000002                E0002                   E   
8                     00000002                E0003                   E   
15                    00000003                E0001                   E   
22                    00000003                E0002                   E   
29                    00000003                E0003                   E   
...                        ...                  ...                 ...   
1972                      TASK                E0002                   E   
1979                      TASK                E0003                   E   
1986                      TASK                E0004                   E   
1993                      TEST                E0001                   E   
2000                      WARN                E0001                   E   

     TJ30T_Textual_Status TJ30T_Long_Textual_Status  
1                     LKD                    

#### Read TJ02T data

This table contains all status starting by a **I** in the column **ISTAT**.

Keep only english version.

In [12]:
# Needed columns
tj02t_columns = {
    'ISTAT': 'TJ02T_User_Status_ID',
    'SPRAS': 'TJ02T_User_Language',
    'TXT04': 'TJ02T_Textual_Status',
    'TXT30': 'TJ02T_Long_Textual_Status'
}

# Read the TJ02T file
tj02t_df = read_csv_file("TJ02T", tj02t_columns)

# Only keep th english texts
tj02t_df = tj02t_df[tj02t_df["TJ02T_User_Language"] == "E"]
tj02t_df = tj02t_df.drop(columns = ["TJ02T_User_Language"])

print(tj02t_df)

write_mid_level_csv_file(tj02t_df, "tj02t_filtered")

     TJ02T_User_Status_ID TJ02T_Textual_Status     TJ02T_Long_Textual_Status
4                   I0158                 OSTS  Outstanding task(s) exist(s)
11                  I0159                 ATCO           All Tasks Completed
18                  I0160                 NOTI       Notification (internal)
25                  I0161                 NOTE       Notification (external)
32                  I0162                 DEFR             Defects Recording
...                   ...                  ...                           ...
7844                I0038                 CONI      Configuration incomplete
7851                I0042                 PREL            Partially released
7860                I0043                  LKD                        Locked
7868                I0045                 TECO         Technically completed
7876                I0046                 CLSD                        Closed

[1112 rows x 3 columns]


#### Read JSTO data

This table mainly contains the user profile **STSMA** to use for a specific object id i.e. work order defined by **OBJNR**

**OR** for Work orders

**OV** for Operations

In [13]:
# Needed columns
jsto_columns = {
    'OBJNR': 'JSTO_Object_Number',
    'STSMA': 'JSTO_Status_Profile'
}

# Read TJ30T table
jsto_df = read_csv_file('JSTO', jsto_columns)

# Keep rows about Operations (OV) and Work Orders (OR) only
jsto_df = jsto_df[jsto_df["JSTO_Object_Number"].str.startswith("OV") | jsto_df["JSTO_Object_Number"].str.startswith("OR")]

print(jsto_df)

C:\Users\L0551542\AppData\Local\Temp\ipykernel_24412\1343571107.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df =  pd.read_csv('./data/E1P_010_' + file_name + '.csv', delimiter = ';', usecols = columns_dict.keys())


           JSTO_Object_Number JSTO_Status_Profile
0              OR002400035375            PMMTNWOH
1        OV100009028900000001            PMMTNWOO
2        OV100009028900000002            PMMTNWOO
3        OV100009028900000003            PMMTNWOO
5              OR002400035376            PMMTNWOH
...                       ...                 ...
6399937  OV100070992600000002            PMMTNWOO
6399938  OV100070992600000003            PMMTNWOO
6399939  OV100070992700000001            PMMTNWOO
6399940  OV100070992700000002            PMMTNWOO
6399941  OV100070992700000003            PMMTNWOO

[2214660 rows x 2 columns]


#### Read JEST data (E1P_JEST_OR_V1 and E1P_JEST_OV_V1 files)

This table mainly contains all current status **STAT** for a specific object id i.e. work order defined by **OBJNR**

Only status starting with an **E** are kept. **I** status are not processed.

In [14]:
# Needed columns
jest_columns = {
    'OBJNR': 'JEST_Object_Number',
    'STAT': 'JEST_Object_Status'
}

# Read TJ30T table
jest_or_df = read_csv_file('JEST_OR_V1', jest_columns) # Work order
jest_ov_df = read_csv_file('JEST_OV_V1', jest_columns) # Operations

jest_df = pd.concat([jest_or_df, jest_ov_df])

# Keep only stats starting by E and not I or something else
#jest_df = jest_df[jest_df["JEST_Object_Status"].str.startswith("E")]

print(jest_df)

           JEST_Object_Number JEST_Object_Status
0              OR002900001515              E0003
1              OR002900001515              E0017
2              OR002900001515              I0002
3              OR002900001515              I0016
4              OR002900001515              I0028
...                       ...                ...
1539085  OV100051535800000001              E0002
1539086  OV100051535800000001              E0004
1539087  OV100051535800000001              I0002
1539088  OV100051536000000001              E0001
1539089  OV100051536000000001              I0045

[2728779 rows x 2 columns]


### Merge tables

#### Merge JEST and JSTO tables

In [15]:
df_status_merged = jest_df.merge(jsto_df, how = 'left', left_on = ['JEST_Object_Number'], right_on = ['JSTO_Object_Number'])

In [16]:
df_status_merged = df_status_merged.drop(columns=["JSTO_Object_Number"])

print(df_status_merged)

           JEST_Object_Number JEST_Object_Status JSTO_Status_Profile
0              OR002900001515              E0003            PMMTNWOH
1              OR002900001515              E0017            PMMTNWOH
2              OR002900001515              I0002            PMMTNWOH
3              OR002900001515              I0016            PMMTNWOH
4              OR002900001515              I0028            PMMTNWOH
...                       ...                ...                 ...
2728774  OV100051535800000001              E0002            PMMTNWOO
2728775  OV100051535800000001              E0004            PMMTNWOO
2728776  OV100051535800000001              I0002            PMMTNWOO
2728777  OV100051536000000001              E0001            PMMTNWOO
2728778  OV100051536000000001              I0045            PMMTNWOO

[2728779 rows x 3 columns]


#### Merge df_status_merged and TJ30T table

Get **status** corresponding to **status code** and **user profile**.

In [17]:
df_status_merged = df_status_merged.merge(tj30t_df, how = "left", left_on = ['JSTO_Status_Profile', 'JEST_Object_Status'], right_on = ['TJ30T_User_Status_Profile', 'TJ30T_User_Status_ID'])

In [18]:
print(df_status_merged[df_status_merged["JEST_Object_Number"] == "OR002100058590"])

       JEST_Object_Number JEST_Object_Status JSTO_Status_Profile  \
513697     OR002100058590              E0007            PMMTNWOH   
513698     OR002100058590              E0030            PMMTNWOH   
513699     OR002100058590              I0007            PMMTNWOH   
513700     OR002100058590              I0009            PMMTNWOH   
513701     OR002100058590              I0016            PMMTNWOH   
513702     OR002100058590              I0028            PMMTNWOH   
513703     OR002100058590              I0046            PMMTNWOH   
513704     OR002100058590              I0215            PMMTNWOH   
513705     OR002100058590              I0321            PMMTNWOH   
513706     OR002100058590              I0420            PMMTNWOH   

       TJ30T_User_Status_Profile TJ30T_User_Status_ID TJ30T_User_Language  \
513697                  PMMTNWOH                E0007                   E   
513698                  PMMTNWOH                E0030                   E   
513699              

#### Merge df_status_merged and TJ02T Table

Get **I Status** corresponding to **I status** code.

In [19]:
df_status_merged = df_status_merged.merge(tj02t_df, how = "left", left_on = ['JEST_Object_Status'], right_on = ['TJ02T_User_Status_ID'])

In [20]:
print(df_status_merged[df_status_merged["JEST_Object_Number"] == "OR002100058590"])

       JEST_Object_Number JEST_Object_Status JSTO_Status_Profile  \
513697     OR002100058590              E0007            PMMTNWOH   
513698     OR002100058590              E0030            PMMTNWOH   
513699     OR002100058590              I0007            PMMTNWOH   
513700     OR002100058590              I0009            PMMTNWOH   
513701     OR002100058590              I0016            PMMTNWOH   
513702     OR002100058590              I0028            PMMTNWOH   
513703     OR002100058590              I0046            PMMTNWOH   
513704     OR002100058590              I0215            PMMTNWOH   
513705     OR002100058590              I0321            PMMTNWOH   
513706     OR002100058590              I0420            PMMTNWOH   

       TJ30T_User_Status_Profile TJ30T_User_Status_ID TJ30T_User_Language  \
513697                  PMMTNWOH                E0007                   E   
513698                  PMMTNWOH                E0030                   E   
513699              

### Filter and Rename columns to get the final Work Order Status and Operation status merges

Columns kept:
* Work Orders status
  * JEST_Object_Number -> WO_Object_Number
  * JEST_Object_Status -> WO_Status_ID
  * JSTO_Status_Profile -> WO_Status_Profile
  * TJ30T_Textual_Status -> WO_Status_Code
  * TJ30T_Long_Textual_Status -> WO_Status_Message
  * TJ02T_Textual_Status -> WO_I_Status_Code
  * TJ02T_Long_Textual_Status -> WO_I_Status_Message

* Operations Status
  * JEST_Object_Number -> OPR_Object_Number
  * JEST_Object_Status -> OPR_Status_ID
  * JSTO_Status_Profile -> OPR_Status_Profile
  * TJ30T_Textual_Status -> OPR_Status_Code
  * TJ30T_Long_Textual_Status -> OPR_Status_Message
  * TJ02T_Textual_Status -> OPR_I_Status_Code
  * TJ02T_Long_Textual_Status -> OPR_I_Status_Message

In [21]:
df_status_merged = df_status_merged[["JEST_Object_Number", "JEST_Object_Status", "JSTO_Status_Profile", "TJ30T_Textual_Status", "TJ30T_Long_Textual_Status", "TJ02T_Textual_Status", "TJ02T_Long_Textual_Status"]]

def get_df(column_names_prefix_value, filter_prefix_value):
    df_local = df_status_merged[df_status_merged["JEST_Object_Number"].str.startswith(filter_prefix_value)]

    df_local.rename(inplace = True, columns = {
        "JEST_Object_Number": column_names_prefix_value + "_Object_Number", 
        "JEST_Object_Status": column_names_prefix_value + "_Status_ID", 
        "JSTO_Status_Profile": column_names_prefix_value + "_Status_Profile", 
        "TJ30T_Textual_Status": column_names_prefix_value + "_E_Status_Code", 
        "TJ30T_Long_Textual_Status": column_names_prefix_value + "_E_Status_Message",
        "TJ02T_Textual_Status": column_names_prefix_value + "_I_Status_Code", 
        "TJ02T_Long_Textual_Status": column_names_prefix_value + "_I_Status_Message"
        }
    )
    df_local.replace(pd.NA, "", inplace = True)

    return df_local

df_wo_status_merged = get_df("WO", "OR")
df_opr_status_merged = get_df("OPR", "OV")

C:\Users\L0551542\AppData\Local\Temp\ipykernel_24412\2758546654.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_local.rename(inplace = True, columns = {
C:\Users\L0551542\AppData\Local\Temp\ipykernel_24412\2758546654.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_local.replace(pd.NA, "", inplace = True)
C:\Users\L0551542\AppData\Local\Temp\ipykernel_24412\2758546654.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df

### Save the file

In [22]:
write_mid_level_csv_file(df_wo_status_merged, "work_orders_status")
write_mid_level_csv_file(df_opr_status_merged, "operations_status")

## 3. Functional location

Tables:
- ILOA_V1
- IFLOS

### Read and format data

#### Read ILOA data

In [23]:
# Needed columns
iloa_columns = {
    'ILOAN': 'ILOAN_Technical_Location_ID',
    'TPLNR': 'ILOAN_Functional_Location_ID',
    'SWERK': 'ILOAN_Location_Plant_Code',
    'MSGRP': 'ILOAN_Location_Room'
}

# Read ILOA table
iloa_df = read_csv_file("ILOA_V1", iloa_columns)

print(iloa_df)

        ILOAN_Technical_Location_ID ILOAN_Functional_Location_ID  \
0                           2512161         ?0100000000000632852   
1                           2512180         ?0100000000000634098   
2                           1180579         ?0100000000000580069   
3                           1180580         ?0100000000000580070   
4                           1180581         ?0100000000000580071   
...                             ...                          ...   
489421                      2668856         ?0100000000000646315   
489422                      2668874         ?0100000000000649833   
489423                      2668873         ?0100000000000649833   
489424                      2668907         ?0100000000000649698   
489425                      2668911         ?0100000000000733468   

       ILOAN_Location_Plant_Code ILOAN_Location_Room  
0                           DK30                 NaN  
1                           DK30                 NaN  
2                 

C:\Users\L0551542\AppData\Local\Temp\ipykernel_24412\1343571107.py:2: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  df =  pd.read_csv('./data/E1P_010_' + file_name + '.csv', delimiter = ';', usecols = columns_dict.keys())


#### Read IFLOS data

In [24]:
# Needed columns
iflos_columns = {
    'TPLNR': 'IFLOS_Functional_Location_ID',
    'STRNO': 'IFLOS_Functional_Location_Name',
}

# Read IFLOS table
iflos_df = read_csv_file("IFLOS", iflos_columns)

print(iflos_df)

       IFLOS_Functional_Location_ID     IFLOS_Functional_Location_Name
0              ?0100000000000594008  HB /A /10/HBA-6-PTS-030-4103-FE-U
1              ?0100000000000594009  HB /A /10/HBA-6-PTS-030-4293-FC-T
2              ?0100000000000594011           HB /A /10/HBAA-XCV-30109
3              ?0100000000000594014           HB /A /10/HBAA-XCV-30158
4              ?0100000000000594016  HB /A /10/HBA-6-PTS-033-4007-FC-U
...                             ...                                ...
241366         ?0100000000000974789       SV /A /86/INSP-OOS-EQUIPMENT
241367         ?0100000000000977560           TW /C /78/TWCD-ZSC-61630
241368         ?0100000000000977561           TW /B /78/TWBD-ZSC-61630
241369         ?0100000000000977562           TE /E /78/TEEE-ZSC-61630
241370         ?0100000000000977563           TE /B /78/TEBD-ZSC-61630

[241371 rows x 2 columns]


### Merge the data - ILOA and IFLOS

In [25]:
df_functional_location_merged = iloa_df.merge(iflos_df, how = "left", left_on = ["ILOAN_Functional_Location_ID"], right_on = ["IFLOS_Functional_Location_ID"])

print(df_functional_location_merged)

        ILOAN_Technical_Location_ID ILOAN_Functional_Location_ID  \
0                           2512161         ?0100000000000632852   
1                           2512180         ?0100000000000634098   
2                           1180579         ?0100000000000580069   
3                           1180580         ?0100000000000580070   
4                           1180581         ?0100000000000580071   
...                             ...                          ...   
504639                      2668856         ?0100000000000646315   
504640                      2668874         ?0100000000000649833   
504641                      2668873         ?0100000000000649833   
504642                      2668907         ?0100000000000649698   
504643                      2668911         ?0100000000000733468   

       ILOAN_Location_Plant_Code ILOAN_Location_Room  \
0                           DK30                 NaN   
1                           DK30                 NaN   
2              

### Filter and Rename columns to get the final Work Order Status merge

Columns kept:
* ILOAN_Technical_Location_ID -> FLOC_Technical_ID
* ILOAN_Functional_Location_ID -> FLOC_Functional_ID
* IFLOS_Functional_Location_Name -> FLOC_Name

In [26]:
# drop useless columns
df_functional_location_merged = df_functional_location_merged[["ILOAN_Technical_Location_ID", "ILOAN_Functional_Location_ID", "IFLOS_Functional_Location_Name", "ILOAN_Location_Room", "ILOAN_Location_Plant_Code"]]

# rename columns
df_functional_location_merged.rename(inplace = True, columns = {
    "ILOAN_Technical_Location_ID": "FLOC_Technical_ID", 
    "ILOAN_Functional_Location_ID": "FLOC_Functional_ID", 
    "IFLOS_Functional_Location_Name": "FLOC_Name",
    "ILOAN_Location_Plant_Code": "FLOC_Plant_Code"
})


# drop duplicates (keep last registered entry for FLOC_Functional_ID)
df_functional_location_merged.drop_duplicates(subset = ['FLOC_Technical_ID', 'FLOC_Functional_ID'], keep = 'last', inplace = True)

# reset index
df_functional_location_merged.reset_index(inplace = True, drop = True)

# drop created index column

print(df_functional_location_merged)

        FLOC_Technical_ID    FLOC_Functional_ID  \
0                 2512161  ?0100000000000632852   
1                 2512180  ?0100000000000634098   
2                 1180579  ?0100000000000580069   
3                 1180580  ?0100000000000580070   
4                 1180581  ?0100000000000580071   
...                   ...                   ...   
489421            2668856  ?0100000000000646315   
489422            2668874  ?0100000000000649833   
489423            2668873  ?0100000000000649833   
489424            2668907  ?0100000000000649698   
489425            2668911  ?0100000000000733468   

                                 FLOC_Name ILOAN_Location_Room FLOC_Plant_Code  
0                    HW /A /07/HWAA-P-5902                 NaN            DK30  
1            HW /A /10/HEA06-MANUAL-VALVES                 NaN            DK30  
2                 HD /A /23/HDAD-BDV-33035                 NaN            DK60  
3                HD /A /23/HDAD-ESDV-33032                 NaN 

### Save the data

In [27]:
write_mid_level_csv_file(df_functional_location_merged, "functional_locations")

## 4. Work Order tables

Tables:
* AFKO
* AUFK
* AFIH
* YTLXXPM_BICNOLTX_V1 (notifications)
* QMEL (notifications)

### Read and format data

#### Read AFIH data

In [28]:
# Needed columns
afih_columns = {
    'AUFNR': 'AFIH_WO_Number', 
    'GEWRK': 'AFIH_WO_WBS_ID', 
    'PRIOK': 'AFIH_WO_Priority', 
    'REVNR': 'AFIH_WO_Revision', 
    'INGPR': 'AFIH_WO_Planner_Group', 
    'ILART': 'AFIH_WO_Activity_Type', 
    'QMNUM': 'AFIH_WO_Notification_Number',
    'ILOAN': 'AFIH_WO_Functional_Location_Number', # Main functional location
    'OBKNR': 'AFIH_WO_Plan_Maintenance_Number', # Secondary functional location
    'WARPL': 'AFIH_WO_Maintenance_Plan_Name',
    'ANLZU': 'AFIH_WO_System_Condition'
}

# Read AFIH table
afih_df = read_csv_file("AFIH_V1", afih_columns)
#print(afih_df)

# Filter on nan worker order id - I don't know if we keep orders without any WBS, the next line does it if needed
#afih_df = afih_df[afih_df['AFIH_WO_WBS_ID'].notna()]
#afih_df = afih_df[afih_df['AFIH_WO_Number'].notna()]

print(afih_df)

        AFIH_WO_Number AFIH_WO_Priority  AFIH_WO_Functional_Location_Number  \
0           2300009145                3                             2558055   
1           2400352096                A                             2610303   
2           2100114335                3                             2571638   
3           2100114433                3                             2604131   
4           2800031347                2                             2529524   
...                ...              ...                                 ...   
121025      2100087418                4                             2273323   
121026      2100086200                2                             2260911   
121027      2100086201                2                             2260913   
121028      2800024552                3                             2260977   
121029      2900000760                3                             2261013   

       AFIH_WO_System_Condition AFIH_WO_Planner_Gro

In [29]:
print(afih_df[afih_df["AFIH_WO_Notification_Number"] == 1100025915])

       AFIH_WO_Number AFIH_WO_Priority  AFIH_WO_Functional_Location_Number  \
78306      2100027283                4                             1374014   

      AFIH_WO_System_Condition AFIH_WO_Planner_Group  AFIH_WO_WBS_ID  \
78306                        C                   MFM        10001300   

      AFIH_WO_Maintenance_Plan_Name  AFIH_WO_Plan_Maintenance_Number  \
78306                           NaN                           143847   

      AFIH_WO_Revision AFIH_WO_Activity_Type  AFIH_WO_Notification_Number  
78306           GOFRYS                    DE                 1.100026e+09  


#### Read AFKO data

In [30]:
# Needed columns
afko_columns = {
    'AUFNR': 'AFKO_WO_Number',
    'AUFPL': 'AFKO_WO_Operation_ID',
    'GSTRS': 'AFKO_WO_Scheduled_Start_Date',
    'GSTRP': "AFKO_WO_Basic_Start_Date",
    'GLTRP': "AFKO_WO_Basic_End_Date",
    'AUFNT': "AFKO_WO_SubNetwork_ID"
}

# Read AFKO table
afko_df = read_csv_file("AFKO_V1", afko_columns)
print(afko_df)


       AFKO_WO_Number  AFKO_WO_Basic_End_Date  AFKO_WO_Basic_Start_Date  \
0          2300008768                20270418                  20240501   
1          2300008769                20270418                  20240501   
2          2300008770                20270418                  20240501   
3          2300008771                20270418                  20240501   
4          2300008772                20270418                  20240501   
...               ...                     ...                       ...   
591456     2400266965                20230727                  20230727   
591457     2400266966                20240110                  20230807   
591458     2400266967                20230630                  20230630   
591459     2400266971                20230816                  20230816   
591460     2400266972                20230822                  20230822   

        AFKO_WO_Scheduled_Start_Date  AFKO_WO_Operation_ID  \
0                           20240501 

C:\Users\L0551542\AppData\Local\Temp\ipykernel_24412\1343571107.py:2: DtypeWarning: Columns (2,47) have mixed types. Specify dtype option on import or set low_memory=False.
  df =  pd.read_csv('./data/E1P_010_' + file_name + '.csv', delimiter = ';', usecols = columns_dict.keys())


#### Read AUFK data

In [31]:
# Needed columns
aufk_columns = {
    'AUFNR': 'AUFK_WO_Number', 
    'KTEXT': 'AUFK_WO_Header_Description', 
    'AUART': 'AUFK_WO_Order_Type', 
    'PHAS0': 'AUFK_WO_Phase_Order_Created', 
    'PHAS1': 'AUFK_WO_Phase_Order_Released', 
    'ZZ_OLAFD': 'AUFK_WO_Original_Deadline', 
    'ZZ_EASD': 'AUFK_WO_Earliest_Allowed_Start_Date', 
    'ZZ_LAFD': 'AUFK_WO_Latest_Allowed_Finish_Date', 
    'OBJNR': 'AUFK_WO_Object_Number'
}

# Read AUFK table
aufk_df = read_csv_file("AUFK_V1", aufk_columns)
print(aufk_df)

       AUFK_WO_Number AUFK_WO_Order_Type  \
0        002900001591                WSO   
1        002900001587                WSO   
2        002900001515                WSO   
3        002900001542                WSO   
4        002900001585                WSO   
...               ...                ...   
121590     2300005429                WRO   
121591     2300005420                WRO   
121592     2300005425                WRO   
121593     2300005428                WRO   
121594     2900000516                WSO   

                     AUFK_WO_Header_Description AUFK_WO_Phase_Order_Created  \
0                       Rolf Scheduled WHM 2024                         NaN   
1                 DFB01,Plug retrieval,WL,19862                         NaN   
2                        Scheduled WHM SAN 2024                         NaN   
3                         Scheduled WHM 2024 DD                           X   
4                        Scheduled WHM KRA 2024                         NaN 

C:\Users\L0551542\AppData\Local\Temp\ipykernel_24412\1343571107.py:2: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df =  pd.read_csv('./data/E1P_010_' + file_name + '.csv', delimiter = ';', usecols = columns_dict.keys())


#### Read YTLXXPM_BICNOLTX_V1 table

Keep only notifications related to a maintenance work order

In [32]:
# Needed columns
notif_columns = {
    'QMNUM': 'NOTIF_Number',
    'YROWNUM': 'NOTIF_Row_Number',
    'LONGTEXT': 'NOTIF_Row_Text'
}

# Read notifications table
notif_df = read_csv_file("YTLXXPM_BICNOLTX_V1", notif_columns)


# Keep only notifications linked to a work order in AFIH
notif_df = notif_df[notif_df["NOTIF_Number"].isin(afih_df["AFIH_WO_Notification_Number"].unique())]

# Drop duplicated rows
notif_df.drop_duplicates(subset = ["NOTIF_Number", "NOTIF_Row_Number"], inplace = True)

# Sort the df by notif number and row number
notif_df = notif_df.sort_values(["NOTIF_Number", "NOTIF_Row_Number"])

print(notif_df)


          NOTIF_Number  NOTIF_Row_Number  \
1123541     1100025915                 1   
1123542     1100025915                 2   
1123543     1100025915                 3   
1123544     1100025915                 4   
1123545     1100025915                 5   
...                ...               ...   
14757874    1700005935                 1   
14757875    1700005935                 2   
14757876    1700005939                 1   
14757877    1700005939                 2   
14757878    1700005939                 3   

                                             NOTIF_Row_Text  
1123541     .05.2013 05:55:41 Mikael Borre Thomsen (MBT009)  
1123542   oolstykke samt checkventil nedstrøms GC-HCV-09...  
1123543   O ifbm kommende frysejob. Spoolstykke (mat: 20...  
1123544   bpakkelager og ny checkventil bestilles på den...  
1123545                                             bpakke.  
...                                                     ...  
14757874    08.10.2024 15:11:57 CET L

In [33]:
# Group by with string concatenation - each row of the notification text is contained in a row of the file, all linked by the notification number and the row number
notif_df["NOTIF_Row_Text"] = notif_df["NOTIF_Row_Text"].astype(str)
notif_processed_df = notif_df[["NOTIF_Number", "NOTIF_Row_Text"]].groupby(["NOTIF_Number"])["NOTIF_Row_Text"].agg('\n'.join)

print(notif_processed_df)

NOTIF_Number
1100025915    .05.2013 05:55:41 Mikael Borre Thomsen (MBT009...
1100025916    Work to be done\n-06-2009 hras\nr kom pludseli...
1100025917    .02.2018 07:43:39 UTC Morten Jensen (MJE099) P...
1100025918    .12.2011 18:41:26 Hans-Jørgen Andreasen (HJA03...
1100025919    .03.2012 10:35:35 Frank Lemmiche (FLE008)\n co...
                                    ...                        
1700005928    04.10.2024 10:25:01 CET Jens Christian SIMONSE...
1700005929    04.10.2024 10:26:14 CET Jens Christian SIMONSE...
1700005933    08.10.2024 12:21:44 CET Rene GRINDERSLEV (L051...
1700005935    08.10.2024 15:11:57 CET Lars NIELSEN (J0516237...
1700005939    09.10.2024 13:14:04 CET Nis Hogh LARSEN (L0516...
Name: NOTIF_Row_Text, Length: 53063, dtype: object


In [34]:
print(notif_processed_df[1100025915])

.05.2013 05:55:41 Mikael Borre Thomsen (MBT009)
oolstykke samt checkventil nedstrøms GC-HCV-090006 skal udskiftes til
O ifbm kommende frysejob. Spoolstykke (mat: 200227751) sendt på
bpakkelager og ny checkventil bestilles på denne WO ligeledes til
bpakke.
rk order closed as part of Backlog rationalization
oject - Ref. NM1800007341


In [35]:
# save mid file
write_mid_level_csv_file(notif_processed_df, "notif_processed")

#### Read QMEL data

Keep only notifications related to a maintenance work order

In [36]:
# Needed columns
qmel_columns = {
    'QMNUM': 'QMEL_Notification_Number',
    'QMDAT': 'QMEL_Malfunction_Started',
    'ERDAT': 'QMEL_Notification_Created'
}

# Read notifications table
qmel_df = read_csv_file("QMEL_V1", qmel_columns)


# Keep only notifications linked to a work order in AFIH
qmel_df = qmel_df[qmel_df["QMEL_Notification_Number"].isin(afih_df["AFIH_WO_Notification_Number"].unique())]

print(qmel_df)

        QMEL_Notification_Number  QMEL_Notification_Created  \
0                     1400352484                   20240426   
1                     1100147446                   20240426   
2                     1100147453                   20240426   
3                     1100147461                   20240426   
4                     1100147462                   20240426   
...                          ...                        ...   
107039                1400284084                   20230501   
107040                1400284090                   20230501   
107041                1100118367                   20230501   
107042                1100117478                   20230419   
107044                1100117500                   20230420   

        QMEL_Malfunction_Started  
0                       20240426  
1                       20240426  
2                       20240426  
3                       20240426  
4                       20240426  
...                          ... 

### Merge the data

#### Merge AFIH and AFKO

In [37]:
# Merge AFKO and AUFK
df_work_order_merged = afih_df.merge(afko_df, how = 'left', left_on = ['AFIH_WO_Number'], right_on = ['AFKO_WO_Number'])

print(df_work_order_merged)

       AFIH_WO_Number AFIH_WO_Priority  AFIH_WO_Functional_Location_Number  \
0          2300009145                3                             2558055   
1          2400352096                A                             2610303   
2          2100114335                3                             2571638   
3          2100114433                3                             2604131   
4          2800031347                2                             2529524   
...               ...              ...                                 ...   
121025     2100087418                4                             2273323   
121026     2100086200                2                             2260911   
121027     2100086201                2                             2260913   
121028     2800024552                3                             2260977   
121029     2900000760                3                             2261013   

       AFIH_WO_System_Condition AFIH_WO_Planner_Group  AFIH_WO_

#### Merge AUFK and df_work_order_merged

In [38]:
# Merge AFKO and AUFK
df_work_order_merged = df_work_order_merged.merge(aufk_df, how = 'left', left_on = ['AFIH_WO_Number'], right_on = ['AUFK_WO_Number'])

print(df_work_order_merged)

       AFIH_WO_Number AFIH_WO_Priority  AFIH_WO_Functional_Location_Number  \
0          2300009145                3                             2558055   
1          2400352096                A                             2610303   
2          2100114335                3                             2571638   
3          2100114433                3                             2604131   
4          2800031347                2                             2529524   
...               ...              ...                                 ...   
121025     2100087418                4                             2273323   
121026     2100086200                2                             2260911   
121027     2100086201                2                             2260913   
121028     2800024552                3                             2260977   
121029     2900000760                3                             2261013   

       AFIH_WO_System_Condition AFIH_WO_Planner_Group  AFIH_WO_

In [39]:
print(df_work_order_merged.columns)

Index(['AFIH_WO_Number', 'AFIH_WO_Priority',
       'AFIH_WO_Functional_Location_Number', 'AFIH_WO_System_Condition',
       'AFIH_WO_Planner_Group', 'AFIH_WO_WBS_ID',
       'AFIH_WO_Maintenance_Plan_Name', 'AFIH_WO_Plan_Maintenance_Number',
       'AFIH_WO_Revision', 'AFIH_WO_Activity_Type',
       'AFIH_WO_Notification_Number', 'AFKO_WO_Number',
       'AFKO_WO_Basic_End_Date', 'AFKO_WO_Basic_Start_Date',
       'AFKO_WO_Scheduled_Start_Date', 'AFKO_WO_Operation_ID',
       'AFKO_WO_SubNetwork_ID', 'AUFK_WO_Number', 'AUFK_WO_Order_Type',
       'AUFK_WO_Header_Description', 'AUFK_WO_Phase_Order_Created',
       'AUFK_WO_Phase_Order_Released', 'AUFK_WO_Object_Number',
       'AUFK_WO_Original_Deadline', 'AUFK_WO_Latest_Allowed_Finish_Date',
       'AUFK_WO_Earliest_Allowed_Start_Date'],
      dtype='object')


#### Merge df_work_order_merged and notifications

In [40]:
df_work_order_merged = df_work_order_merged.merge(notif_processed_df, how = 'left', left_on = ['AFIH_WO_Notification_Number'], right_on = ['NOTIF_Number'])

print(df_work_order_merged)

       AFIH_WO_Number AFIH_WO_Priority  AFIH_WO_Functional_Location_Number  \
0          2300009145                3                             2558055   
1          2400352096                A                             2610303   
2          2100114335                3                             2571638   
3          2100114433                3                             2604131   
4          2800031347                2                             2529524   
...               ...              ...                                 ...   
121025     2100087418                4                             2273323   
121026     2100086200                2                             2260911   
121027     2100086201                2                             2260913   
121028     2800024552                3                             2260977   
121029     2900000760                3                             2261013   

       AFIH_WO_System_Condition AFIH_WO_Planner_Group  AFIH_WO_

#### Merge df_work_order_merged and QMEL

In [41]:
df_work_order_merged = df_work_order_merged.merge(qmel_df, how = 'left', left_on = ['AFIH_WO_Notification_Number'], right_on = ['QMEL_Notification_Number'])

print(df_work_order_merged)

       AFIH_WO_Number AFIH_WO_Priority  AFIH_WO_Functional_Location_Number  \
0          2300009145                3                             2558055   
1          2400352096                A                             2610303   
2          2100114335                3                             2571638   
3          2100114433                3                             2604131   
4          2800031347                2                             2529524   
...               ...              ...                                 ...   
121025     2100087418                4                             2273323   
121026     2100086200                2                             2260911   
121027     2100086201                2                             2260913   
121028     2800024552                3                             2260977   
121029     2900000760                3                             2261013   

       AFIH_WO_System_Condition AFIH_WO_Planner_Group  AFIH_WO_

#### Filter and Rename columns to get the final Work Orders merge

Columns kept:
* AFIH_WO_Number -> WO_Number
* AFIH_WO_Priority -> WO_Priority
* AFIH_WO_Functional_Location_Number -> WO_Functional_Location_Number
* AFIH_WO_Plan_Maintenance_Number -> WO_Plan_Maintenance_Number
* AFIH_WO_Planner_Group -> WO_Planner_Group
* AFIH_WO_WBS_ID -> WO_WBS_ID
* AFIH_WO_Revision -> WO_Revision
* AFIH_WO_Activity_Type -> WO_Activity_Type
* AFKO_WO_Scheduled_Start_Date -> WO_Scheduled_Start_Date
* AFKO_WO_Operation_ID -> WO_Operation_ID
* AUFK_WO_Order_Type -> WO_Order_Type
* AUFK_WO_Header_Description -> WO_Header_Description
* AUFK_WO_Phase_Order_Created -> WO_Phase_Order_Created 
* AUFK_WO_Phase_Order_Released -> WO_Phase_Order_Released
* AUFK_WO_Object_Number -> WO_Status_ID
* AUFK_WO_Original_Deadline -> WO_Original_Deadline
* AFIH_WO_Notification_Number -> WO_Notification_Number
* NOTIF_Row_Text -> WO_Notification
* QMEL_Malfunction_Started -> WO_Notification_Malfunction_Started
* QMEL_Notification_Created -> WO_Notification_Created
* AFIH_WO_Maintenance_Plan_Name -> WO_Maintenance_Plan_Name
* AFIH_WO_System_Condition -> WO_System_Condition
* AFKO_WO_Basic_Start_Date -> WO_Basic_Start_Date
* AFKO_WO_Basic_End_Date -> WO_Basic_End_Date
* AUFK_WO_Earliest_Allowed_Start_Date -> WO_Earliest_Allowed_Start_Date
* AUFK_WO_Latest_Allowed_Finish_Date -> WO_Latest_Allowed_Finish_Date
* AFKO_WO_SubNetwork_ID -> WO_SubNetwork_ID

In [42]:
# drop useless columns
df_work_order_merged = df_work_order_merged[["AFIH_WO_Number", "AFIH_WO_Priority", "AFIH_WO_Functional_Location_Number", "AFIH_WO_Plan_Maintenance_Number",
                                             "AFIH_WO_Planner_Group", "AFIH_WO_WBS_ID", "AFIH_WO_Revision", "AFIH_WO_Activity_Type", "AFKO_WO_Scheduled_Start_Date", 
                                             "AFKO_WO_Operation_ID", "AUFK_WO_Order_Type", "AUFK_WO_Header_Description", "AUFK_WO_Phase_Order_Created", 
                                             "AUFK_WO_Phase_Order_Released", "AUFK_WO_Object_Number", "AUFK_WO_Original_Deadline", "AFIH_WO_Notification_Number", 
                                             "QMEL_Malfunction_Started", "QMEL_Notification_Created", "NOTIF_Row_Text", "AFIH_WO_Maintenance_Plan_Name", "AFIH_WO_System_Condition",
                                             "AFKO_WO_Basic_Start_Date", "AFKO_WO_Basic_End_Date", "AUFK_WO_Earliest_Allowed_Start_Date", "AUFK_WO_Latest_Allowed_Finish_Date", 
                                             "AFKO_WO_SubNetwork_ID"
                                             ]]

# rename columns
df_work_order_merged.rename(inplace = True, columns = {
    "AFIH_WO_Number": "WO_Number", 
    "AFIH_WO_Priority": "WO_Priority", 
    "AFIH_WO_Functional_Location_Number": "WO_Functional_Location_Number",
    "AFIH_WO_Plan_Maintenance_Number": "WO_Plan_Maintenance_Number",
    "AFIH_WO_Planner_Group": "WO_Planner_Group",
    "AFIH_WO_WBS_ID": "WO_WBS_ID",
    "AFIH_WO_Revision": "WO_Revision",
    "AFIH_WO_Activity_Type": "WO_Activity_Type",
    "AFKO_WO_Scheduled_Start_Date": "WO_Scheduled_Start_Date",
    "AFKO_WO_Operation_ID": "WO_Operation_ID",
    "AUFK_WO_Order_Type": "WO_Order_Type",
    "AUFK_WO_Header_Description": "WO_Header_Description",
    "AUFK_WO_Phase_Order_Created": "WO_Phase_Order_Created",
    "AUFK_WO_Phase_Order_Released": "WO_Phase_Order_Released",
    "AUFK_WO_Object_Number": "WO_Status_ID",
    "AUFK_WO_Original_Deadline": "WO_Original_Deadline",
    "AFIH_WO_Notification_Number": "WO_Notification_Number",
    "QMEL_Malfunction_Started": "WO_Notification_Malfunction_Started",
    "QMEL_Notification_Created": "WO_Notification_Created",
    "AFIH_WO_Maintenance_Plan_Name": "WO_Maintenance_Plan_Name",
    "AFIH_WO_System_Condition": "WO_System_Condition",
    "NOTIF_Row_Text": "WO_Notification",
    "AFKO_WO_Basic_Start_Date": "WO_Basic_Start_Date",
    "AFKO_WO_Basic_End_Date": "WO_Basic_End_Date",
    "AUFK_WO_Earliest_Allowed_Start_Date": "WO_Earliest_Allowed_Start_Date",
    "AUFK_WO_Latest_Allowed_Finish_Date": "WO_Latest_Allowed_Finish_Date",
    "AFKO_WO_SubNetwork_ID": "WO_SubNetwork_ID"
})

print(df_work_order_merged)

         WO_Number WO_Priority  WO_Functional_Location_Number  \
0       2300009145           3                        2558055   
1       2400352096           A                        2610303   
2       2100114335           3                        2571638   
3       2100114433           3                        2604131   
4       2800031347           2                        2529524   
...            ...         ...                            ...   
121025  2100087418           4                        2273323   
121026  2100086200           2                        2260911   
121027  2100086201           2                        2260913   
121028  2800024552           3                        2260977   
121029  2900000760           3                        2261013   

        WO_Plan_Maintenance_Number WO_Planner_Group  WO_WBS_ID WO_Revision  \
0                           572308              MIN   10001172         NaN   
1                           580945              INP   10001416 

### save data into a csv file

In [43]:
write_mid_level_csv_file(df_work_order_merged, "work_orders")

## 5. Secondary functional locations

It's possible to perform the secondary functional locations only after the work order processing to avoid loading the same table multiple times

Table:
* OBJK

### Read OBJK data

In [44]:
# Needed columns
objk_columns = {
    'OBKNR': 'PM_Object_Number',
    'SORTF': 'PM_Object_Sorting',
    'OBJVW': 'PM_Object_Usage',
    'ILOAN': 'PM_Functional_Location'
}

# Read AFIH table
objk_df = read_csv_file("OBJK", objk_columns)

# Filter on NaN function location
objk_df.dropna(subset = ["PM_Functional_Location"], inplace = True)
objk_df.reset_index(inplace = True, drop = True)

print(objk_df)

         PM_Object_Number  PM_Functional_Location PM_Object_Sorting  \
0                  572073               2127560.0              0020   
1                  572084               2126675.0              0010   
2                  572084               2127974.0              0020   
3                  572084               2125953.0              0030   
4                  572084               2125961.0              0040   
...                   ...                     ...               ...   
1468164            598358               2608974.0               NaN   
1468165            598367               2608974.0               NaN   
1468166            598271               2609038.0               NaN   
1468167            598377               2609038.0               NaN   
1468168            598270               2609038.0               NaN   

        PM_Object_Usage  
0                     A  
1                     A  
2                     A  
3                     A  
4                

C:\Users\L0551542\AppData\Local\Temp\ipykernel_24412\1343571107.py:2: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  df =  pd.read_csv('./data/E1P_010_' + file_name + '.csv', delimiter = ';', usecols = columns_dict.keys())


### Save data

In [45]:
write_mid_level_csv_file(objk_df, 'secondary_locations')

In [46]:
print(objk_df[objk_df["PM_Object_Number"] == 572084])

   PM_Object_Number  PM_Functional_Location PM_Object_Sorting PM_Object_Usage
1            572084               2126675.0              0010               A
2            572084               2127974.0              0020               A
3            572084               2125953.0              0030               A
4            572084               2125961.0              0040               A
5            572084               2126363.0              0050               A
6            572084               2126274.0              0060               A


## 6. Operations Tables

Tables:
* AFVC
* AFVV

### Read data

#### Read AFVC data

In [47]:
# needed columns
afvc_columns = {
    "ARBID": 'AFVC_WBS_ID',
    "AUFPL": 'AFVC_Operation_Routing_Number',
    "VORNR": "AFVC_Operation_Activity_Number",
    "APLZL": 'AFVC_Operation_Counter',
    "ANZZL": 'AFVC_Workers_Number',
    "ABLAD": "AFVC_Scheduled_Work",
    "OBJNR": "AFVC_Status_ID",
    "LTXA1": "AFVC_Operation_Short_Text"
}

# read the df
afvc_df = read_csv_file("AFVC_V1", afvc_columns)

In [48]:
print(afvc_df)

        AFVC_Operation_Routing_Number  AFVC_Operation_Counter  \
0                          1000589075                       3   
1                          1000589075                       4   
2                          1000589075                       5   
3                          1000589075                       6   
4                          1000589075                       7   
...                               ...                     ...   
518445                     1000515357                       1   
518446                     1000515358                       1   
518447                     1000515359                       1   
518448                     1000515360                       1   
518449                     1000197463                      13   

        AFVC_Operation_Activity_Number  AFVC_WBS_ID  \
0                                   30     10001079   
1                                   40     10001080   
2                                   50     10001081   

#### Read AFVV data

In [49]:
# needed columns
afvv_columns = {
    "AUFPL": 'AFVV_Operation_Routing_Number',
    "APLZL": 'AFVV_Operation_Counter',
    "ARBEI": 'AFVV_Planned_Work',
    "ISMNW": 'AFVV_Actual_Work',
    "FSSBD": 'AFVV_Work_Start_Date',
    "FSSBZ": 'AFVV_Work_Start_Time',
    "FSSAD": 'AFVV_Work_End_Date',
    "FSSAZ": 'AFVV_Work_End_Time'
}

# read the df
afvv_df = read_csv_file("AFVV_V1", afvv_columns)

In [50]:
print(afvv_df)

        AFVV_Operation_Routing_Number  AFVV_Operation_Counter  \
0                          1000589075                       3   
1                          1000589075                       4   
2                          1000589075                       5   
3                          1000589075                       6   
4                          1000589075                       7   
...                               ...                     ...   
518445                     1000197463                      13   
518446                     1000515357                       1   
518447                     1000515358                       1   
518448                     1000515359                       1   
518449                     1000515360                       1   

        AFVV_Planned_Work  AFVV_Actual_Work  AFVV_Work_Start_Date  \
0                     2.0               0.0              20241010   
1                     6.0               0.0              20241011   
2           

### Merge AFVC and AFVV

In [51]:
df_operations_merged = afvc_df.merge(afvv_df, how = 'left', left_on = ["AFVC_Operation_Routing_Number", "AFVC_Operation_Counter"], right_on = ["AFVV_Operation_Routing_Number", "AFVV_Operation_Counter"])

print(df_operations_merged)

        AFVC_Operation_Routing_Number  AFVC_Operation_Counter  \
0                          1000589075                       3   
1                          1000589075                       4   
2                          1000589075                       5   
3                          1000589075                       6   
4                          1000589075                       7   
...                               ...                     ...   
518445                     1000515357                       1   
518446                     1000515358                       1   
518447                     1000515359                       1   
518448                     1000515360                       1   
518449                     1000197463                      13   

        AFVC_Operation_Activity_Number  AFVC_WBS_ID  \
0                                   30     10001079   
1                                   40     10001080   
2                                   50     10001081   

In [52]:
print(df_operations_merged[df_operations_merged["AFVC_Operation_Routing_Number"] == 1000126414])

        AFVC_Operation_Routing_Number  AFVC_Operation_Counter  \
58551                      1000126414                       7   
101912                     1000126414                       1   
101913                     1000126414                       2   
101914                     1000126414                       3   
101915                     1000126414                       4   
101916                     1000126414                       5   
101917                     1000126414                       6   

        AFVC_Operation_Activity_Number  AFVC_WBS_ID  \
58551                               40     10001289   
101912                              10     10001293   
101913                              11     10001289   
101914                              12     10001288   
101915                              20     10001289   
101916                              25     10001293   
101917                              30     10001324   

           AFVC_Operation_Short_Text  

### Filter and Rename columns to get the final Operations merge

Columns kept:
* AFVC_Operation_Routing_Number -> WO_OPR_Routing_Number
* AFVC_Operation_Counter -> WO_OPR_Counter
* AFVC_Operation_Activity_Number -> OPR_Activity_Number
* AFVC_WBS_ID -> WO_OPR_WBS_ID
* AFVC_Workers_Number -> WO_OPR_Workers_Numbers
* AFVV_Planned_Work -> OPR_Planned_Work
* AFVV_Actual_Work -> OPR_Actual_Work
* AFVV_Work_Start_Date -> OPR_Start_Date
* AFVV_Work_Start_Time -> OPR_Start_Time
* AFVV_Work_End_Date -> OPR_End_Date
* AFVV_Work_End_Time -> OPR_End_Time
* AFVC_Scheduled_Work -> OPR_Scheduled_Work
* AFVC_Status_ID -> OPR_Status_ID
* AFVC_Operation_Short_Text -> OPR_Description

In [53]:
# drop useless columns
df_operations_merged = df_operations_merged[["AFVC_Operation_Routing_Number", "AFVC_Operation_Counter", "AFVC_WBS_ID", 
                                             "AFVC_Workers_Number", "AFVV_Planned_Work", "AFVV_Actual_Work", "AFVV_Work_Start_Date", "AFVV_Work_Start_Time",
                                             "AFVV_Work_End_Date", "AFVV_Work_End_Time", "AFVC_Scheduled_Work", "AFVC_Operation_Short_Text", 
                                             "AFVC_Operation_Activity_Number", "AFVC_Status_ID"
                                            ]]

# rename columns
df_operations_merged.rename(inplace = True, columns = {
    "AFVC_Operation_Routing_Number": "OPR_Routing_Number", 
    "AFVC_Operation_Counter": "OPR_Counter", 
    "AFVC_Operation_Activity_Number": "OPR_Activity_Number",
    "AFVC_WBS_ID": "OPR_WBS_ID",
    "AFVC_Workers_Number": "OPR_Workers_Numbers",
    "AFVV_Planned_Work": "OPR_Planned_Work",
    "AFVV_Actual_Work": "OPR_Actual_Work",
    "AFVV_Work_Start_Date": "OPR_Start_Date",
    "AFVV_Work_Start_Time": "OPR_Start_Time",
    "AFVV_Work_End_Date": "OPR_End_Date",
    "AFVV_Work_End_Time": "OPR_End_Time",
    "AFVC_Scheduled_Work": "OPR_Scheduled_Work",
    "AFVC_Status_ID": "OPR_Status_ID",
    "AFVC_Operation_Short_Text": "OPR_Description",
})


print(df_operations_merged)

        OPR_Routing_Number  OPR_Counter  OPR_WBS_ID  OPR_Workers_Numbers  \
0               1000589075            3    10001079                    1   
1               1000589075            4    10001080                    2   
2               1000589075            5    10001081                    2   
3               1000589075            6    10001080                    2   
4               1000589075            7    10001079                    1   
...                    ...          ...         ...                  ...   
518445          1000515357            1    10001440                    0   
518446          1000515358            1    10001168                    0   
518447          1000515359            1           0                    0   
518448          1000515360            1    10001080                    0   
518449          1000197463           13    10001566                    0   

        OPR_Planned_Work  OPR_Actual_Work  OPR_Start_Date  OPR_Start_Time  \
0         

In [54]:
write_mid_level_csv_file(df_operations_merged, "work_operations")

## Functions to get all data related to one Work Order

### Read mid csv files

Simulates API getting its data from read csv files

In [55]:
def read_mid_csv_file(file_name):
    print("-----------------------------------")
    print("-----------------------------------")
    print("-----------------------------------")
    print("------ " + file_name + " ----------")
    print("\n")
    df = pd.read_csv('./data/mid_' + file_name + '.csv', delimiter = ',', index_col=0)
    print(df)
    return df

# Locations
functional_locations_df = read_mid_csv_file("functional_locations")
plan_maintenance_location_df = read_mid_csv_file("secondary_locations")
plan_maintenance_location_df["PM_Functional_Location"] = plan_maintenance_location_df["PM_Functional_Location"].astype(int)

# Work order
work_orders_df = read_mid_csv_file("work_orders")

# Operations
operations_df = read_mid_csv_file("work_operations")

# WBS
wbs_df = read_mid_csv_file("work_center")

# Status
wo_status_df = read_mid_csv_file("work_orders_status")
opr_status_df = read_mid_csv_file("operations_status")

-----------------------------------
-----------------------------------
-----------------------------------
------ functional_locations ----------




C:\Users\L0551542\AppData\Local\Temp\ipykernel_24412\859204161.py:7: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./data/mid_' + file_name + '.csv', delimiter = ',', index_col=0)


        FLOC_Technical_ID    FLOC_Functional_ID  \
0                 2512161  ?0100000000000632852   
1                 2512180  ?0100000000000634098   
2                 1180579  ?0100000000000580069   
3                 1180580  ?0100000000000580070   
4                 1180581  ?0100000000000580071   
...                   ...                   ...   
489421            2668856  ?0100000000000646315   
489422            2668874  ?0100000000000649833   
489423            2668873  ?0100000000000649833   
489424            2668907  ?0100000000000649698   
489425            2668911  ?0100000000000733468   

                                 FLOC_Name ILOAN_Location_Room FLOC_Plant_Code  
0                    HW /A /07/HWAA-P-5902                 NaN            DK30  
1            HW /A /10/HEA06-MANUAL-VALVES                 NaN            DK30  
2                 HD /A /23/HDAD-BDV-33035                 NaN            DK60  
3                HD /A /23/HDAD-ESDV-33032                 NaN 

C:\Users\L0551542\AppData\Local\Temp\ipykernel_24412\859204161.py:7: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./data/mid_' + file_name + '.csv', delimiter = ',', index_col=0)


         PM_Object_Number  PM_Functional_Location PM_Object_Sorting  \
0                  572073               2127560.0              0020   
1                  572084               2126675.0              0010   
2                  572084               2127974.0              0020   
3                  572084               2125953.0              0030   
4                  572084               2125961.0              0040   
...                   ...                     ...               ...   
1468164            598358               2608974.0               NaN   
1468165            598367               2608974.0               NaN   
1468166            598271               2609038.0               NaN   
1468167            598377               2609038.0               NaN   
1468168            598270               2609038.0               NaN   

        PM_Object_Usage  
0                     A  
1                     A  
2                     A  
3                     A  
4                

C:\Users\L0551542\AppData\Local\Temp\ipykernel_24412\859204161.py:7: DtypeWarning: Columns (13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./data/mid_' + file_name + '.csv', delimiter = ',', index_col=0)


         WO_Number WO_Priority  WO_Functional_Location_Number  \
0       2300009145           3                        2558055   
1       2400352096           A                        2610303   
2       2100114335           3                        2571638   
3       2100114433           3                        2604131   
4       2800031347           2                        2529524   
...            ...         ...                            ...   
121025  2100087418           4                        2273323   
121026  2100086200           2                        2260911   
121027  2100086201           2                        2260913   
121028  2800024552           3                        2260977   
121029  2900000760           3                        2261013   

        WO_Plan_Maintenance_Number WO_Planner_Group  WO_WBS_ID WO_Revision  \
0                           572308              MIN   10001172         NaN   
1                           580945              INP   10001416 

### Some tests

In [56]:
### filter work order status df on status SCH
filtered_wo_status_df = wo_status_df[wo_status_df["WO_E_Status_Code"] == "SCH"]
wo_status_id = filtered_wo_status_df.WO_Object_Number.unique() 

### filter work order using work order status id of wo having SCH
filtered_wo_df = work_orders_df[work_orders_df["WO_Status_ID"].isin(wo_status_id)]
print(filtered_wo_df)


         WO_Number WO_Priority  WO_Functional_Location_Number  \
923     2100110750           3                        2528698   
4436    2100061900           4                        1939617   
9106    2400344885           B                        2558002   
9114    2400350944           A                        2602608   
9119    2400337337           C                        2529898   
...            ...         ...                            ...   
120791  2800024486           4                        2259821   
120792  2800024489           4                        2259825   
120793  2800024495           4                        2259836   
120822  2800024369           4                        2258521   
120824  2800024373           4                        2258535   

        WO_Plan_Maintenance_Number WO_Planner_Group  WO_WBS_ID WO_Revision  \
923                         559589              MFM   10001036         NaN   
4436                        332984              MFM   10001444 

In [57]:
look_for_location = "?0100000000000647856"

# filter functional location df
filtered_functional_location_df = functional_locations_df[functional_locations_df["FLOC_Functional_ID"] == look_for_location]

print(filtered_functional_location_df)

# get functional location of work order 2100035861
filtered_wo_df = work_orders_df[work_orders_df["WO_Functional_Location_Number"].isin(filtered_functional_location_df["FLOC_Technical_ID"].unique())]

print(filtered_wo_df["WO_Number"].unique())

# 

        FLOC_Technical_ID    FLOC_Functional_ID            FLOC_Name  \
69317             1255398  ?0100000000000647856  GO /C /78/GC-M-710A   
144225            1348374  ?0100000000000647856  GO /C /78/GC-M-710A   
144415            1365335  ?0100000000000647856  GO /C /78/GC-M-710A   
145260            1368871  ?0100000000000647856  GO /C /78/GC-M-710A   
145510            1347739  ?0100000000000647856  GO /C /78/GC-M-710A   
...                   ...                   ...                  ...   
478516            2649413  ?0100000000000647856  GO /C /78/GC-M-710A   
478602            2651807  ?0100000000000647856  GO /C /78/GC-M-710A   
483481            2688772  ?0100000000000647856  GO /C /78/GC-M-710A   
487223            2701588  ?0100000000000647856  GO /C /78/GC-M-710A   
487225            2701587  ?0100000000000647856  GO /C /78/GC-M-710A   

       ILOAN_Location_Room FLOC_Plant_Code  
69317                  NaN            DK80  
144225                 NaN            DK80  


### API-Like functions

In [58]:
def get_int_data(data):
    return str(data).replace(".0", "") if not pd.isnull(data) else "-"

def get_wbs_data(wbs_id):
    # Extract from the wbs df the concerned wbs
    wbs_tmp_df = wbs_df[wbs_df["WBS_ID"] == wbs_id]
    if len(wbs_tmp_df) == 0:
        return "None"
    
    wbs_data = wbs_tmp_df.iloc[0]

    return {
        "ID": wbs_id,
        "Name": wbs_data["WBS_Name"],
        "Plant": wbs_data["WBS_Plant"],
        "Full_Name": wbs_data["WBS_Full_name"]
    }

def get_status_data(status_number: str):

    def get_column_name(is_wo, suffix_column_name):
        return "_".join(["WO" if is_wo else "OPR", suffix_column_name])

    def get_status_filtered_df(is_wo):
        df_to_filter = wo_status_df if is_wo else opr_status_df

        return df_to_filter[df_to_filter[get_column_name(is_wo, "Object_Number")] == status_number]

    # It needs to know if the status is linked to a WO or to an Operation
    is_wo = status_number.startswith("OR")

    # Get status related to the work order
    status_tmp_df = get_status_filtered_df(is_wo)
    
    e_status = []
    i_status = []

    for index, row in status_tmp_df.iterrows():
        if row[get_column_name(is_wo, "Status_ID")].startswith("E"):
            e_status.append({
                "ID": row[get_column_name(is_wo, "Status_ID")],
                "Profile": row[get_column_name(is_wo, "Status_Profile")],
                "Code": row[get_column_name(is_wo, "E_Status_Code")],
                "Message": row[get_column_name(is_wo, "E_Status_Message")],
            })
        elif row[get_column_name(is_wo, "Status_ID")].startswith("I"):
            i_status.append({
                "ID": row[get_column_name(is_wo, "Status_ID")],
                "Profile": row[get_column_name(is_wo, "Status_Profile")],
                "Code": row[get_column_name(is_wo, "I_Status_Code")],
                "Message": row[get_column_name(is_wo, "I_Status_Message")],
            })

    return {
        "Status_ID": status_number,
        "E_Status": e_status,
        "I_Status": i_status
    }

def get_operations_data(wo_operation_id, wo_status):
    # Get operations for the WO
    opr_tmp_df = operations_df[operations_df["OPR_Routing_Number"] == wo_operation_id].sort_values(["OPR_Counter"])

    # Get scheduled data or not depending on the WO status contains SCH
    #is_scheduled = len(filter(lambda stat: stat["Code"] == "SCH", wo_status["E_Status"])) == 1
    is_scheduled = len([status for status in wo_status["E_Status"] if status["Code"] == "SCH"]) == 1

    # Format in json
    res = []
    for index, row in opr_tmp_df.iterrows():
        res.append({
            "Counter": row["OPR_Counter"],
            "Workers_Numbers": row["OPR_Workers_Numbers"],
            "Planned_Work": row["OPR_Planned_Work"],
            "Actual_Work": row["OPR_Actual_Work"],
            "Work_Start_Date": get_int_data(row["OPR_Start_Date"]),
            "Work_Start_Time": get_int_data(row["OPR_Start_Time"]),
            "Work_End_Date": get_int_data(row["OPR_End_Date"]),
            "Work_End_Time": get_int_data(row["OPR_End_Time"]),
            "Scheduled_Work": row["OPR_Scheduled_Work"] if is_scheduled else "-",
            "Work_Center": get_wbs_data(row["OPR_WBS_ID"]),
            "Status": get_status_data(row["OPR_Status_ID"]),
            "Operation_Short_Text": row["OPR_Description"]
        })

    return res

def get_location_data(location_number):
    loc_tmp_df = functional_locations_df[functional_locations_df["FLOC_Technical_ID"] == location_number]
    # Extract from the functional location df the concerned location
    if len(loc_tmp_df) == 0:
        return "None"
    elif len(loc_tmp_df) > 1:
        raise Exception("Too much locations have been found for this id ", location_number)
    
    loc_data = loc_tmp_df.iloc[0]

    #print(loc_data)

    return {
        "ID": loc_data["FLOC_Technical_ID"],
        "Name": loc_data["FLOC_Name"],
        "Plant": loc_data["FLOC_Plant_Code"]
    }

def get_plan_maintenance(plan_number):
    #print(plan_number)
    pm_tmp_df = plan_maintenance_location_df[plan_maintenance_location_df["PM_Object_Number"] == plan_number].merge(functional_locations_df, how = "left", left_on=["PM_Functional_Location"], right_on = ["FLOC_Technical_ID"]).sort_values(["PM_Object_Sorting"])

    secondary_locations = []
    for index, row in pm_tmp_df.iterrows():
        secondary_locations.append({
            "ID": row["FLOC_Technical_ID"],
            "Counter": row["PM_Object_Sorting"],
            "Name": row["FLOC_Name"]
        })

    #print(pm_tmp_df)

    return secondary_locations

def get_work_order_data(work_order_id):

    # Extract from the work order df the concerned wo
    wo_tmp_df = work_orders_df[work_orders_df["WO_Number"] == work_order_id]
    if len(wo_tmp_df) == 0:
        raise Exception("Impossible to pursue, no work order found for this id ", work_order_id)
    
    wo_data = wo_tmp_df.iloc[0]

    #print(wo_data)

    # Operations
    #print(get_operations_data(wo_data["WO_Operation_ID"]))
    # Functional Location
    #print(get_location_data(wo_data["WO_Functional_Location_Number"]))
    # Get WBS
    #print(get_wbs_data(wo_data["WO_WBS_ID"]))
    # Work Order status
    #print(get_status_data(wo_data["WO_Status_ID"]))
    # Plan maintenance - secondary locations
    #print(get_plan_maintenance(wo_data["WO_Plan_Maintenance_Number"]))

    wo_status = get_status_data(wo_data["WO_Status_ID"])
    
    return {
        "ID": wo_data["WO_Number"],
        "Priority": wo_data["WO_Priority"],
        "Planner_Group": wo_data["WO_Planner_Group"],
        "Revision": wo_data["WO_Revision"],
        "Activity_Type": wo_data["WO_Activity_Type"],
        "Order_Type": wo_data["WO_Order_Type"],
        "Scheduled_Start_Date": get_int_data(wo_data["WO_Scheduled_Start_Date"]),
        "Phase_Order_Created": get_int_data(wo_data["WO_Phase_Order_Created"]),
        "Phase_Order_Released": get_int_data(wo_data["WO_Phase_Order_Released"]),
        "Original_Deadline": get_int_data(wo_data["WO_Original_Deadline"]),
        "Header_Description": wo_data["WO_Header_Description"],
        "Operations_ID": get_int_data(wo_data["WO_Operation_ID"]),
        "Operations": get_operations_data(wo_data["WO_Operation_ID"], wo_status),
        "Status": wo_status,
        "Location": get_location_data(wo_data["WO_Functional_Location_Number"]),
        "Plan_Maintenance": get_plan_maintenance(wo_data["WO_Plan_Maintenance_Number"]),
        "Work_Center": get_wbs_data(wo_data["WO_WBS_ID"]),
        "Notification_Number": get_int_data(wo_data["WO_Notification_Number"]),
        "Notification_Created": get_int_data(wo_data["WO_Notification_Created"]),
        "Notification_Malfunction_Started": get_int_data(wo_data["WO_Notification_Malfunction_Started"]),
        "Notification": wo_data["WO_Notification"],
        "Earliest_Allowed_Start_Date": wo_data["WO_Earliest_Allowed_Start_Date"],
        "Latest_Allowed_Finish_Date": wo_data["WO_Latest_Allowed_Finish_Date"],
        "Basic_Start_Date": wo_data["WO_Basic_Start_Date"],
        "Basic_End_Date": wo_data["WO_Basic_End_Date"],
        "System_Condition": wo_data["WO_System_Condition"],
        "Maintenance_Plan_Name": wo_data["WO_Maintenance_Plan_Name"]
    }

pprint.pprint(get_work_order_data(2100024286))

{'Activity_Type': 'DE',
 'Basic_End_Date': 20210206.0,
 'Basic_Start_Date': 20201109.0,
 'Earliest_Allowed_Start_Date': 20191013.0,
 'Header_Description': 'ANOS HGS19-1-2 GOC module 2 repl. gratin',
 'ID': 2100024286,
 'Latest_Allowed_Finish_Date': 20200311.0,
 'Location': {'ID': 1370910,
              'Name': 'GO /C /86/30375 - GRATINGS',
              'Plant': 'DK80'},
 'Maintenance_Plan_Name': nan,
 'Notification': ' DETAILED FAULT DESCRIPTION(SPECIFICS, MEASSURABLES, '
                 'LOCATION)\n'
                 'nding from handrail and grating survey March 2019\n'
                 'cation:  GOC 30.350 Module 2\n'
                 'nding no.: 1, 3\n'
                 'iticality: 2\n'
                 'nding 1: Grating unsupportet and corroded - needs attention\n'
                 'nding 3: Grating in staircase towards GOD side of GOC worn '
                 'out and\n'
                 'vered with scaffolding plate\n'
                 'ndor from Semco ongoing with fabrication d